In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_217051/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        # self.discard_below_average(sort_by="r2")
        # self.discard_below_average(sort_by="r2_vt")
        # self.discard_high_standard_deviation()
        self.results_df.sort_values(by="mse")
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.8)
analize.Analize()
# analize.clean_folder(subfolder="dataset", extension="pkl")
# analize.clean_folder(subfolder="results", extension="xlsx")
# analize.clean_folder(subfolder="results", extension="txt")
# analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_1_8_3,0.987164,0.645232,0.975290,0.997549,0.987507,0.004204,0.210606,0.008642,0.001035,0.004839,0.028528,0.064838,0.977079,0.065358,120.943484,276.938989,"Hidden Size=[2, 12], regularizer=0.02, learnin..."
1,model_1_9_5,0.987160,0.648674,0.993461,0.982945,0.987997,0.004205,0.208562,0.001636,0.008180,0.004908,0.027698,0.064848,0.977072,0.065369,120.942829,276.938334,"Hidden Size=[2, 12], regularizer=0.02, learnin..."
2,model_1_8_6,0.987114,0.646268,0.967121,0.997461,0.983771,0.004220,0.209991,0.011500,0.001072,0.006286,0.028091,0.064965,0.976989,0.065487,120.935641,276.931145,"Hidden Size=[2, 12], regularizer=0.02, learnin..."
3,model_1_8_8,0.987022,0.646079,0.963711,0.997541,0.982275,0.004251,0.210103,0.012692,0.001038,0.006865,0.027790,0.065197,0.976825,0.065720,120.921387,276.916891,"Hidden Size=[2, 12], regularizer=0.02, learnin..."
4,model_1_8_9,0.986990,0.649097,0.958968,0.997657,0.980197,0.004261,0.208311,0.014351,0.000990,0.007670,0.027103,0.065276,0.976768,0.065800,120.916513,276.912018,"Hidden Size=[2, 12], regularizer=0.02, learnin..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244,model_1_0_4,0.464013,0.261149,0.470168,0.454914,0.469897,0.175545,0.438614,0.139729,0.144823,0.142276,0.219402,0.418981,0.042880,0.422346,113.479721,269.475226,"Hidden Size=[2, 12], regularizer=0.02, learnin..."
245,model_1_0_3,0.428511,0.237412,0.408295,0.438907,0.431576,0.187172,0.452705,0.156047,0.149076,0.152561,0.231289,0.432634,-0.020517,0.436109,113.351449,269.346954,"Hidden Size=[2, 12], regularizer=0.02, learnin..."
246,model_1_0_2,0.345003,0.163290,0.302284,0.339228,0.330156,0.214523,0.496707,0.184004,0.175560,0.179782,0.248141,0.463166,-0.169637,0.466886,113.078681,269.074186,"Hidden Size=[2, 12], regularizer=0.02, learnin..."
247,model_1_0_1,0.331616,0.153729,0.285990,0.346414,0.325707,0.218907,0.502383,0.188301,0.173650,0.180976,0.251252,0.467875,-0.193544,0.471633,113.038214,269.033719,"Hidden Size=[2, 12], regularizer=0.02, learnin..."
